In [1]:

import pandas as pd
from pathlib import Path

excel_path = Path(r"/Users/cheickberthe/PycharmProjects/spreadsheet-analyzer/test_assets/collection/business-accounting/Business Accounting.xlsx")
df = pd.read_excel(excel_path, sheet_name=4)
print(f"Loaded sheet with shape: {df.shape}")

# Quick preview for LLM context
print("\nFirst 10 rows:")
print(df.head(10).to_string())
print("\nLast 5 rows:")
print(df.tail(5).to_string())

# Get basic info
sheet_name = "None"
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"


Loaded sheet with shape: (1, 11)

First 10 rows:
   Date                           Description  Transaction type  USD Amount  XOF Amount  Column 1  Category  TOTAL REVENUES   0  TOTAL EXPENSES  0.1
0   NaN  NC Secretary of State Annual Filings               NaN         NaN         NaN       NaN       NaN             NaN NaN             NaN  NaN

Last 5 rows:
   Date                           Description  Transaction type  USD Amount  XOF Amount  Column 1  Category  TOTAL REVENUES   0  TOTAL EXPENSES  0.1
0   NaN  NC Secretary of State Annual Filings               NaN         NaN         NaN       NaN       NaN             NaN NaN             NaN  NaN


/Users/cheickberthe/PycharmProjects/spreadsheet-analyzer/.venv/lib/python3.12/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [2]:

excel_file_name = excel_path.name
sheet_dimensions = f"{df.shape[0]} rows x {df.shape[1]} columns"
sheet_name


'None'

In [3]:

# Table Detection Analysis
print("=== DATA STRUCTURE ANALYSIS ===")
print(f"DataFrame shape: {df.shape}")
print(f"Columns: {list(df.columns)}")
print("\n=== FIRST 10 ROWS ===")
print(df.head(10).to_string())
print("\n=== LAST 5 ROWS ===")
print(df.tail(5).to_string())

print("\n=== COLUMN ANALYSIS ===")
# Check for empty columns that might separate tables
for i, col in enumerate(df.columns):
    non_null_count = df[col].notna().sum()
    print(f"Column {i} ('{col}'): {non_null_count}/{len(df)} non-null values")

print("\n=== EMPTY ROW ANALYSIS ===")
# Check for empty rows that might separate tables
empty_rows = df.isnull().all(axis=1)
if empty_rows.any():
    empty_row_indices = empty_rows[empty_rows].index.tolist()
    print(f"Empty rows found at indices: {empty_row_indices[:10]}")
else:
    print("No completely empty rows found")

print("\n=== SIDE-BY-SIDE ANALYSIS ===")
# Check for potential side-by-side tables by examining column groups
left_cols = df.iloc[:, 0:7]  # First 7 columns
right_cols = df.iloc[:, 7:]  # Remaining columns
print(f"Left section shape: {left_cols.shape}")
print(f"Right section shape: {right_cols.shape}")
print(f"Left section non-null density: {left_cols.notna().sum().sum() / (left_cols.shape[0] * left_cols.shape[1]):.2f}")
print(f"Right section non-null density: {right_cols.notna().sum().sum() / (right_cols.shape[0] * right_cols.shape[1]):.2f}")


=== DATA STRUCTURE ANALYSIS ===
DataFrame shape: (1, 11)
Columns: ['Date', 'Description', 'Transaction type', 'USD Amount', 'XOF Amount', 'Column 1', 'Category', 'TOTAL REVENUES', 0, 'TOTAL EXPENSES', '0.1']

=== FIRST 10 ROWS ===
   Date                           Description  Transaction type  USD Amount  XOF Amount  Column 1  Category  TOTAL REVENUES   0  TOTAL EXPENSES  0.1
0   NaN  NC Secretary of State Annual Filings               NaN         NaN         NaN       NaN       NaN             NaN NaN             NaN  NaN

=== LAST 5 ROWS ===
   Date                           Description  Transaction type  USD Amount  XOF Amount  Column 1  Category  TOTAL REVENUES   0  TOTAL EXPENSES  0.1
0   NaN  NC Secretary of State Annual Filings               NaN         NaN         NaN       NaN       NaN             NaN NaN             NaN  NaN

=== COLUMN ANALYSIS ===
Column 0 ('Date'): 0/1 non-null values
Column 1 ('Description'): 1/1 non-null values
Column 2 ('Transaction type'): 0/1 non-nul

In [4]:
detected_tables = []

# Given the data structure and analysis, it appears there is a primary table occupying columns 0-6 (indices 0-6) with data, and a secondary, possibly empty or summary section in columns 7-10.
# The shape of the DataFrame is (1, 11), indicating only one row of data, which is likely a header or a placeholder.
# Since no data rows are present beyond the header, and the only row is mostly empty, we consider the header as a potential table boundary.
# The main table likely spans from row 0 to 0 (single header row), columns 0 to 6.
# The secondary section in columns 7-10 appears to be empty or not containing data.

# Define the main data table
detected_tables.append({
    "table_id": "table_1",
    "description": "Main financial data table with transaction details",
    "start_row": 0,
    "end_row": 0,
    "start_col": 0,
    "end_col": 6,
    "confidence": 0.9,
    "table_type": "HEADER",
    "entity_type": "financial_transactions"
})

# Since the right section is empty or non-informative, no additional tables are detected.

# Final output
detected_tables

[{'table_id': 'table_1',
  'description': 'Main financial data table with transaction details',
  'start_row': 0,
  'end_row': 0,
  'start_col': 0,
  'end_col': 6,
  'confidence': 0.9,
  'table_type': 'HEADER',
  'entity_type': 'financial_transactions'}]

In [5]:

# Extract detection results created by LLM
# The detector should have created a 'detected_tables' variable with the results

# CLAUDE-TEST-WORKAROUND: Validate that detected_tables exists and is properly formatted
if 'detected_tables' in globals():
    # Validate it's a list
    if isinstance(detected_tables, list):
        detection_results = detected_tables
        print(f"✅ Found {len(detection_results)} tables from LLM detection")
        # Validate first table has required fields (if any tables exist)
        if detection_results:
            required_fields = ['table_id', 'description', 'start_row', 'end_row', 'start_col', 'end_col']
            first_table = detection_results[0]
            missing_fields = [f for f in required_fields if f not in first_table]
            if missing_fields:
                print(f"⚠️ Warning: First table missing fields: {missing_fields}")
    else:
        print(f"❌ Error: detected_tables is not a list, it's a {type(detected_tables)}")
        detection_results = []
else:
    # CLAUDE-GOTCHA: Gemini sometimes fails to create the variable even after multiple prompts
    print("❌ No 'detected_tables' variable found - LLM failed to complete detection")
    detection_results = []

detection_results


✅ Found 1 tables from LLM detection


[{'table_id': 'table_1',
  'description': 'Main financial data table with transaction details',
  'start_row': 0,
  'end_row': 0,
  'start_col': 0,
  'end_col': 6,
  'confidence': 0.9,
  'table_type': 'HEADER',
  'entity_type': 'financial_transactions'}]


# Table Detection Results

Detected 1 tables:


## Table 1: Main financial data table with transaction details
- Location: Rows 0-0, Columns 0-6
- Type: header
- Entity: financial_transactions
- Confidence: 0.90
